In [1]:
#提取不满意的意见
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import jieba.analyse
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
import jieba.posseg as pseg
from tqdm import tqdm
import collections

In [2]:
dfe = pd.read_csv('..\data\data_E.csv',index_col = 0,dtype = {'simple review':str,'review':str,})

In [3]:
dfe

,car type,purch time,purch place,purch price,avg rate,publish time,score,purch purp,simple review,review,cum mileage
0,2015款奔驰E级改款 E 260L 运动型,2015-07-01,[河南省],42万 (厂商指导价 46.80万),9.9L/100km,2019-04-09,"{'外观': '4', '内饰': '3', '空间': '4', '舒适': '4', '...","['上班', '接送小孩', '跑长途', '商务接送']",4年的老车了，就拿最新款的比吧，外观我反而喜欢我的老款，新款的尾灯胎小气了。总体还是可以的，...,[最满意]\n放到现在依然不错甚至自认为超过现款的外观！奔驰总是这样，大改款小改款总是改的让...,NaN
1,2017款奔驰E级E 200 运动版,2017-10-26,重庆商社麒兴汽车销售服务有限公司 [重庆市],40万 (厂商指导价 42.28万),9.2L/100km,2018-11-20,"{'外观': '5', '内饰': '5', '空间': '4', '舒适': '4', '...","['上班', '购物', '接送小孩', '自驾游', '商务接送']",标准轴距的车身尺寸比例非常协调，运动版外观线条非常动感。内饰氛围的豪华感十足，中控的大屏很震...,[最满意]\n1、标准轴距的车身尺寸比例非常协调，运动版外观线条非常动感，引擎盖两条肌肉棱线...,NaN
2,2014款奔驰E级改款 E 260L 运动版,2014-01-01,北京之星汽车服务有限公司 [北京市],36.5万 (厂商指导价 44.50万),9.5L/100km,2018-08-28,"{'外观': '3', '内饰': '5', '空间': '4', '舒适': '4', '...","['上班', '购物', '接送小孩', '商务接送']",奔驰的质感在同年份的车里数一数二，尤其内饰的奢华感，基本可以让你分分钟签单买车。行驶质感，隔...,[最满意]\n内饰质感好，当初买车的时候，主要在XTS，5系和E级之间犹豫。5系内饰实在接受...,NaN
3,2018款奔驰E级改款 E 300 L 运动豪华型,2018-07-10,[北京市],45.3万 (厂商指导价 50.28万),11L/100km,2018-08-01,"{'外观': '5', '内饰': '5', '空间': '5', '舒适': '5', '...","['上班', '接送小孩', '拉货']",整体都还可以，就是轮毂太丑了，减配COMMAND系统不能接受。据说轮胎容易坏，目前正在测试中...,[最满意]\n内饰，改了AMG包围以后，外观也很霸气。后排和后窗遮阳帘提升了很大的档次。大标...,NaN
4,2017款奔驰E级E 200L 4MATIC 运动型,2017-02-05,x上海东华之星汽车维修服务有限公司宝山分公司 [上海市],56万 (厂商指导价 45.98万),9.6L/100km,2017-02-09,"{'外观': '5', '内饰': '5', '空间': '5', '舒适': '5', '...","['上班', '商务接送']",总之买了不后悔，很不错，新E的产生真的是完全碾压了其他竞争对手,[最满意]\n外观霸气非常，不认识的人都以为是迈巴赫，不用说，面子感十足 内饰也是用了S级的...,NaN
5,2016款奔驰E级E 320L 4MATIC,2016-01-06,宿迁文峰伟恒汽车销售服务有限公司 [江苏省],49万 (厂商指导价 万),12L/100km,2016-09-02,"{'外观': '5', '内饰': '5', '空间': '5', '舒适': '5', '...",['上班'],NaN,[最满意]\n动力，3.0v6双涡轮的发动机 ，价格也算不错。\n[不满意]\n小毛病多 刚...,NaN
6,2015款奔驰E级E 260L 运动型,2016-04-08,[山东省],41万 (厂商指导价 46.80万),10.3L/100km,2016-08-03,"{'外观': '5', '内饰': '5', '空间': '5', '舒适': '5', '...",['商务接送'],NaN,[最满意]\n商务接送有面子，空间大，真的很大，特别是后排，毕竟是奔驰，虽然是二手车但是经过...,NaN
7,2020款奔驰E级E 260 L 4MATIC 运动型,2020-01-01,天津市北信中乒之星汽车销售服务有限公司 [天津市],44.2万 (厂商指导价 45.28万),10L/100km,2020-04-20,"{'外观': '4', '内饰': '5', '空间': '5', '舒适': '5', '...","['上班', '接送小孩']",奔驰的质感在同年份的车里数一数二，尤其内饰的奢华感，基本可以让你分分钟签单买车。行驶质感，隔...,[最满意]\n内饰质感好，当初买车的时候，主要在XTS，5系和E级之间犹豫。5系内饰实在接受...,NaN
8,2020款奔驰E级E 260 L 4MATIC 运动型,2020-01-01,北京博瑞祥驰汽车销售服务有限公司 [北京市],44.3万 (厂商指导价 45.28万),9L/100km,2020-04-20,"{'外观': '5', '内饰': '4', '空间': '4', '舒适': '4', '...","['上班', '接送小孩']",标准轴距的车身尺寸比例非常协调，运动版外观线条非常动感。内饰氛围的豪华感十足，中控的大屏很震...,[最满意]\n1、标准轴距的车身尺寸比例非常协调，运动版外观线条非常动感，引擎盖两条肌肉棱线...,NaN
9,2020款奔驰E级E 260 L 运动型,2020-01-01,[陕西省],43万 (厂商指导价 42.98万),11L/100km,2020-04-10,"{'外观': '5', '内饰': '5', '空间': '5', '舒适': '5', '...","['上班', '接送小孩']",内饰，改了AMG包围以后，外观也很霸气。后排和后窗遮阳帘提升了很大的档次。大标的三叉戟，和立...,[最满意]\n内饰，改了AMG包围以后，外观也很霸气。后排和后窗遮阳帘提升了很大的档次。大标...,NaN


In [4]:
sum(pd.notnull(dfe['review'])) # 1708 lines with review
dfe_review = dfe[pd.notnull(dfe['review'])] 
dfe_review = dfe_review.reset_index(drop = True)
dfe_review

1708

,car type,purch time,purch place,purch price,avg rate,publish time,score,purch purp,simple review,review,cum mileage
0,2015款奔驰E级改款 E 260L 运动型,2015-07-01,[河南省],42万 (厂商指导价 46.80万),9.9L/100km,2019-04-09,"{'外观': '4', '内饰': '3', '空间': '4', '舒适': '4', '...","['上班', '接送小孩', '跑长途', '商务接送']",4年的老车了，就拿最新款的比吧，外观我反而喜欢我的老款，新款的尾灯胎小气了。总体还是可以的，...,[最满意]\n放到现在依然不错甚至自认为超过现款的外观！奔驰总是这样，大改款小改款总是改的让...,NaN
1,2017款奔驰E级E 200 运动版,2017-10-26,重庆商社麒兴汽车销售服务有限公司 [重庆市],40万 (厂商指导价 42.28万),9.2L/100km,2018-11-20,"{'外观': '5', '内饰': '5', '空间': '4', '舒适': '4', '...","['上班', '购物', '接送小孩', '自驾游', '商务接送']",标准轴距的车身尺寸比例非常协调，运动版外观线条非常动感。内饰氛围的豪华感十足，中控的大屏很震...,[最满意]\n1、标准轴距的车身尺寸比例非常协调，运动版外观线条非常动感，引擎盖两条肌肉棱线...,NaN
2,2014款奔驰E级改款 E 260L 运动版,2014-01-01,北京之星汽车服务有限公司 [北京市],36.5万 (厂商指导价 44.50万),9.5L/100km,2018-08-28,"{'外观': '3', '内饰': '5', '空间': '4', '舒适': '4', '...","['上班', '购物', '接送小孩', '商务接送']",奔驰的质感在同年份的车里数一数二，尤其内饰的奢华感，基本可以让你分分钟签单买车。行驶质感，隔...,[最满意]\n内饰质感好，当初买车的时候，主要在XTS，5系和E级之间犹豫。5系内饰实在接受...,NaN
3,2018款奔驰E级改款 E 300 L 运动豪华型,2018-07-10,[北京市],45.3万 (厂商指导价 50.28万),11L/100km,2018-08-01,"{'外观': '5', '内饰': '5', '空间': '5', '舒适': '5', '...","['上班', '接送小孩', '拉货']",整体都还可以，就是轮毂太丑了，减配COMMAND系统不能接受。据说轮胎容易坏，目前正在测试中...,[最满意]\n内饰，改了AMG包围以后，外观也很霸气。后排和后窗遮阳帘提升了很大的档次。大标...,NaN
4,2017款奔驰E级E 200L 4MATIC 运动型,2017-02-05,x上海东华之星汽车维修服务有限公司宝山分公司 [上海市],56万 (厂商指导价 45.98万),9.6L/100km,2017-02-09,"{'外观': '5', '内饰': '5', '空间': '5', '舒适': '5', '...","['上班', '商务接送']",总之买了不后悔，很不错，新E的产生真的是完全碾压了其他竞争对手,[最满意]\n外观霸气非常，不认识的人都以为是迈巴赫，不用说，面子感十足 内饰也是用了S级的...,NaN
5,2016款奔驰E级E 320L 4MATIC,2016-01-06,宿迁文峰伟恒汽车销售服务有限公司 [江苏省],49万 (厂商指导价 万),12L/100km,2016-09-02,"{'外观': '5', '内饰': '5', '空间': '5', '舒适': '5', '...",['上班'],NaN,[最满意]\n动力，3.0v6双涡轮的发动机 ，价格也算不错。\n[不满意]\n小毛病多 刚...,NaN
6,2015款奔驰E级E 260L 运动型,2016-04-08,[山东省],41万 (厂商指导价 46.80万),10.3L/100km,2016-08-03,"{'外观': '5', '内饰': '5', '空间': '5', '舒适': '5', '...",['商务接送'],NaN,[最满意]\n商务接送有面子，空间大，真的很大，特别是后排，毕竟是奔驰，虽然是二手车但是经过...,NaN
7,2020款奔驰E级E 260 L 4MATIC 运动型,2020-01-01,天津市北信中乒之星汽车销售服务有限公司 [天津市],44.2万 (厂商指导价 45.28万),10L/100km,2020-04-20,"{'外观': '4', '内饰': '5', '空间': '5', '舒适': '5', '...","['上班', '接送小孩']",奔驰的质感在同年份的车里数一数二，尤其内饰的奢华感，基本可以让你分分钟签单买车。行驶质感，隔...,[最满意]\n内饰质感好，当初买车的时候，主要在XTS，5系和E级之间犹豫。5系内饰实在接受...,NaN
8,2020款奔驰E级E 260 L 4MATIC 运动型,2020-01-01,北京博瑞祥驰汽车销售服务有限公司 [北京市],44.3万 (厂商指导价 45.28万),9L/100km,2020-04-20,"{'外观': '5', '内饰': '4', '空间': '4', '舒适': '4', '...","['上班', '接送小孩']",标准轴距的车身尺寸比例非常协调，运动版外观线条非常动感。内饰氛围的豪华感十足，中控的大屏很震...,[最满意]\n1、标准轴距的车身尺寸比例非常协调，运动版外观线条非常动感，引擎盖两条肌肉棱线...,NaN
9,2020款奔驰E级E 260 L 运动型,2020-01-01,[陕西省],43万 (厂商指导价 42.98万),11L/100km,2020-04-10,"{'外观': '5', '内饰': '5', '空间': '5', '舒适': '5', '...","['上班', '接送小孩']",内饰，改了AMG包围以后，外观也很霸气。后排和后窗遮阳帘提升了很大的档次。大标的三叉戟，和立...,[最满意]\n内饰，改了AMG包围以后，外观也很霸气。后排和后窗遮阳帘提升了很大的档次。大标...,NaN


In [41]:
def loadDataset(line):
    corpus = ''
    #加载停用词表
    stop = [line.strip().decode('utf-8') for line in open('..\data\stop_words_4.txt','rb').readlines() ]
    #无外观内饰空间舒适油耗动力操控配置性价比
    segs = jieba.lcut(line.strip(), cut_all=True)   #, cut_all=True
    segs = pseg.cut(line)
    final = ''
    for seg, flag in segs:
        #去停用词
        if seg not in stop:
            #去数词和去字符串
            if flag !='m' and flag !='x':
                    #输出分词
                final +=' '+ seg 
#             corpus.append(final)
    corpus += ' ' +final 
            
    return corpus
# dataset = loadDataset()

In [49]:
def tfidf(dataset):
    tv=TfidfVectorizer(min_df=0.,max_df=1.,norm='l2',use_idf=True,smooth_idf=True)
    tv_matrix=tv.fit_transform([dataset])
    #print(tv_matrix)
    tv_matrix=tv_matrix.toarray()
    #print(tv_matrix)
    vocab=tv.get_feature_names()
    df_com = pd.DataFrame(np.round(tv_matrix,2),columns=vocab).T.sort_values(by=[0],ascending=False)
    df_com.columns = ['tf-idf']
    return df_com[0:50]

In [43]:
total = ''
for i in dfe_review.review.unique():
    total += i
dataset = loadDataset(total)

In [50]:
tfidf(dataset)

,tf-idf
空间,0.41
内饰,0.34
油耗,0.32
动力,0.31
外观,0.27
舒适,0.23
操控,0.22
后排,0.18
配置,0.18
座椅,0.14


In [64]:
tfidf(dataset).index
top = ['空间', '内饰', '油耗', '动力', '外观', '舒适', '操控', '后排', '配置', '座椅', '高速', '性价比',
       '舒服', '驾驶', '舒适性', '发动机', '方向盘', '设计', '运动', '做工', '加速', '大灯', '大气',
       '市区', '油门', '转向', '车身', '功能', '豪华', '起步', '前排', '后备箱', '时尚', '价格', '乘坐',
       '全景', '天窗', '轮胎', '黑色', '刹车', '品牌', '轮毂', '优惠', '提速', '优雅', '氛围', '精致']

Index(['空间', '内饰', '油耗', '动力', '外观', '舒适', '操控', '后排', '配置', '座椅', '高速', '性价比',
       '舒服', '驾驶', '舒适性', '发动机', '方向盘', '设计', '运动', '做工', '加速', '大灯', '大气',
       '市区', '油门', '转向', '车身', '功能', '豪华', '起步', '前排', '后备箱', '时尚', '价格', '乘坐',
       '全景', '天窗', '轮胎', '黑色', '刹车', '品牌', '当然', '轮毂', '优惠', '需要', '使用', '提速',
       '优雅', '氛围', '精致'],
      dtype='object')

In [65]:
# top = tfidf(dataset).index

total_string = []
for i in dfe_review.review.tolist():
    have_string = {}
    for j in top:
        if j in i:
            have_string[i.index(j)] = j
    string = []
    for k in sorted(have_string.keys()):
        string.append(have_string[k])
    total_string.append(string)

In [66]:
df_tfidf = pd.DataFrame(columns=['words'])
df_tfidf['words'] = total_string
df_tfidf

,words
0,"[外观, 发动机, 内饰, 配置, 大灯, 转向, 运动, 轮毂, 座椅, 空间, 氛围, ..."
1,"[车身, 运动, 外观, 内饰, 氛围, 豪华, 座椅, 驾驶, 功能, 操控, 轮毂, 轮..."
2,"[内饰, 价格, 动力, 性价比, 发动机, 外观, 豪华, 品牌, 空间, 后排, 座椅,..."
3,"[内饰, 外观, 后排, 运动, 轮毂, 轮胎, 功能, 后备箱, 豪华, 座椅, 空间, ..."
4,"[外观, 内饰, 做工, 轮胎, 大灯, 豪华, 空间, 前排, 后排, 舒适, 座椅, 舒..."
5,"[动力, 发动机, 价格, 座椅, 大灯, 外观, 运动, 大气, 轮毂, 配置, 内饰, ..."
6,"[空间, 后排, 外观, 时尚, 运动, 大气, 内饰, 做工, 座椅, 舒适, 舒适性, ..."
7,"[内饰, 价格, 动力, 性价比, 发动机, 外观, 豪华, 品牌, 空间, 后排, 座椅,..."
8,"[车身, 运动, 外观, 内饰, 氛围, 豪华, 座椅, 驾驶, 功能, 操控, 轮毂, 轮..."
9,"[内饰, 外观, 后排, 运动, 轮毂, 轮胎, 豪华, 座椅, 空间, 舒适, 舒服, 油..."


In [67]:
keywords_array=[]
for index, row in df_tfidf.iterrows():
    keywords=row['words']
    for kw in keywords:
        keywords_array.append((kw.strip(' '), row['words']))
kw_df = pd.DataFrame(keywords_array).rename(columns={0:'keyword', 1:'keywords'})
kw_df

,keyword,keywords
0,外观,"[外观, 发动机, 内饰, 配置, 大灯, 转向, 运动, 轮毂, 座椅, 空间, 氛围, ..."
1,发动机,"[外观, 发动机, 内饰, 配置, 大灯, 转向, 运动, 轮毂, 座椅, 空间, 氛围, ..."
2,内饰,"[外观, 发动机, 内饰, 配置, 大灯, 转向, 运动, 轮毂, 座椅, 空间, 氛围, ..."
3,配置,"[外观, 发动机, 内饰, 配置, 大灯, 转向, 运动, 轮毂, 座椅, 空间, 氛围, ..."
4,大灯,"[外观, 发动机, 内饰, 配置, 大灯, 转向, 运动, 轮毂, 座椅, 空间, 氛围, ..."
5,转向,"[外观, 发动机, 内饰, 配置, 大灯, 转向, 运动, 轮毂, 座椅, 空间, 氛围, ..."
6,运动,"[外观, 发动机, 内饰, 配置, 大灯, 转向, 运动, 轮毂, 座椅, 空间, 氛围, ..."
7,轮毂,"[外观, 发动机, 内饰, 配置, 大灯, 转向, 运动, 轮毂, 座椅, 空间, 氛围, ..."
8,座椅,"[外观, 发动机, 内饰, 配置, 大灯, 转向, 运动, 轮毂, 座椅, 空间, 氛围, ..."
9,空间,"[外观, 发动机, 内饰, 配置, 大灯, 转向, 运动, 轮毂, 座椅, 空间, 氛围, ..."


In [68]:
document = kw_df.keywords.tolist()
names = kw_df.keyword.tolist()

document_array = []
for item in tqdm(document):
    document_array.append((item))

occurrences = collections.OrderedDict((name, collections.OrderedDict((name, 0) for name in names)) for name in tqdm(names))

# Find the co-occurrences:
for l in tqdm(document_array):
    for i in range(len(l)):
        for item in l[:i] + l[i + 1:]:
            occurrences[l[i]][item] += 1

co_occur = pd.DataFrame.from_dict(occurrences )

100%|██████████████████████████████████████████████████████████████████████████| 31776/31776 [00:05<00:00, 5956.71it/s]


In [69]:
co_occur.to_csv('..\data\co-occurancy_matrix_E.csv')

In [70]:
co_occur

,外观,发动机,内饰,配置,大灯,转向,运动,轮毂,座椅,空间,...,价格,市区,加速,大气,时尚,天窗,方向盘,精致,黑色,全景
乘坐,7329,3543,7362,6002,1921,2160,3496,1814,4645,7390,...,1647,2623,2921,2769,1673,1718,2599,1489,1495,1590
价格,8185,4050,8205,7097,2807,2521,3400,1363,5181,8203,...,0,2559,3118,3413,2403,1701,3324,1620,1164,1652
优惠,9014,3932,8995,8051,3313,2962,4527,2620,5734,8980,...,3377,2602,4103,3426,2602,2227,3558,2713,2307,1970
优雅,6028,2599,6011,4665,2620,2798,3308,1883,4480,6039,...,2162,1940,1759,2308,1341,1453,2594,1601,966,1262
做工,11768,4776,11870,9412,3886,4216,5459,2744,7757,11840,...,3607,3549,3887,4335,2689,2586,4362,2326,1780,2340
全景,6312,2828,6304,5995,2937,2576,3299,2101,4461,6363,...,1652,1898,2235,2198,1457,5612,2590,1268,1462,0
内饰,30807,12618,0,24693,9241,9401,13513,6935,18939,31032,...,8205,10448,10864,10928,6228,6812,11324,5858,4856,6304
刹车,6503,3231,6602,5213,1866,2126,2858,1751,4241,6605,...,2186,2202,2802,2199,1596,1620,3282,1329,1189,1394
前排,8729,3358,8796,6045,2778,2963,4361,1750,7096,8827,...,2286,3010,2806,3000,1857,1564,3536,1881,1386,1589
功能,8099,3285,8120,7015,2991,2830,3886,2114,5308,8168,...,2225,3053,3189,3081,1613,2487,3964,1329,1567,2105
